### Import Library

In [60]:
# Import library yang diperlukan
import os
import random
from sklearn.model_selection import train_test_split
import cv2
import numpy as np


#ignore warning
import warnings
warnings.filterwarnings('ignore')

### Direktori

In [61]:
K1 = 'D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/K1/'
K2 = 'D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/K2/'
K3 = 'D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/K3/'
K4 = 'D:/UTY/Semester 7/Pengembangan Aplikasi AI/Pengenal-Citra-Nomor-Kendaraan/Dataset/K4/'

### Split Data

In [62]:
# Tentukan jalur ke direktori dataset
dataset_dir = K1

# Buat daftar semua berkas citra dalam direktori dataset
berkas_citra = [os.path.join(dataset_dir, berkas) for berkas in os.listdir(dataset_dir) if berkas.endswith('.JPG')]

In [63]:
# Tetapkan nilai benih acak untuk konsistensi
random.seed(42)

# Tentukan rasio data untuk pelatihan dan pengujian
rasio_pelatihan = 0.8  # 80% untuk pelatihan, 20% untuk pengujian

# Bagi dataset menjadi set pelatihan dan pengujian
berkas_pelatihan, berkas_pengujian = train_test_split(berkas_citra, train_size=rasio_pelatihan, test_size=1-rasio_pelatihan)

In [64]:
# Buat direktori untuk dataset pelatihan dan pengujian jika belum ada
direktori_pelatihan = os.path.join(dataset_dir, 'pelatihan')
direktori_pengujian = os.path.join(dataset_dir, 'pengujian')
os.makedirs(direktori_pelatihan, exist_ok=True)
os.makedirs(direktori_pengujian, exist_ok=True)

# Pindahkan berkas ke direktori yang sesuai
for berkas in berkas_pelatihan:
    tujuan = os.path.join(direktori_pelatihan, os.path.basename(berkas))
    os.rename(berkas, tujuan)

for berkas in berkas_pengujian:
    tujuan = os.path.join(direktori_pengujian, os.path.basename(berkas))
    os.rename(berkas, tujuan)

# Cetak jumlah citra dalam masing-masing dataset
print(f"Total citra: {len(berkas_citra)}")
print(f"Citra pelatihan: {len(berkas_pelatihan)}")
print(f"Citra pengujian: {len(berkas_pengujian)}")

Total citra: 237
Citra pelatihan: 189
Citra pengujian: 48


### Preprocessing

In [70]:
# Tentukan jalur ke direktori dataset yang telah dibagi menjadi pelatihan dan pengujian
direktori_pengujian = K1+'pengujian/'
direktori_pelatihan = K1+'pelatihan/'

# Tentukan direktori penyimpanan citra yang telah diproses
direktori_pengujian_hasil = K1+'hasil/pengujian/'
direktori_pelatihan_hasil = K1+'hasil/pelatihan/'

In [99]:
# Fungsi untuk melakukan preprocessing citra
def preprocess_citra(berkas_citra):
    # Baca citra dari berkas
    citra = cv2.imread(berkas_citra)
    
    # Konversi citra ke skala abu-abu
    citra = cv2.cvtColor(citra, cv2.COLOR_BGR2GRAY)
    
    # Konversi citra menjadi blur
    citra = cv2.GaussianBlur(citra, (5, 5), 0)
    
    # Konversi citra menjadi B & W
    citra = cv2.Canny(citra, 100, 200)
    
    # Mengubah ukuran citra menjadi lebih kecil
    ukuran_citra = (500, 500)
    citra = cv2.resize(citra, ukuran_citra)
    
    # # Ubah tipe data menjadi float32
    # citra = citra.astype(np.float32) / 255.0  

    
    # Kembalikan citra yang telah diproses
    return citra


In [100]:
# Preprocess citra dalam direktori pelatihan
for nama_berkas in os.listdir(direktori_pelatihan):
    berkas_citra = os.path.join(direktori_pelatihan, nama_berkas)
    citra_preprocessed = preprocess_citra(berkas_citra)
    
    # Simpan citra yang telah diproses ke direktori yang sama
    nama_berkas_simpan = os.path.join(direktori_pelatihan_hasil, f'preprocessed_{nama_berkas}')
    cv2.imwrite(nama_berkas_simpan, citra_preprocessed)

In [ ]:

# Preprocess citra dalam direktori pengujian
for nama_berkas in os.listdir(direktori_pengujian):
    berkas_citra = os.path.join(direktori_pengujian, nama_berkas)
    citra_preprocessed = preprocess_citra(berkas_citra)
    
    # Simpan citra yang telah diproses ke direktori yang sama
    nama_berkas_simpan = os.path.join(direktori_pengujian_hasil, f'preprocessed_{nama_berkas}')
    cv2.imwrite(nama_berkas_simpan, citra_preprocessed)


# Mode Deteksi Plat Nomor

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# Fungsi untuk memuat dataset dan anotasinya
def load_dataset(direktori):
    images = []
    labels = []
    
    for nama_berkas in os.listdir(direktori):
        berkas_citra = os.path.join(direktori, nama_berkas)
        citra = cv2.imread(berkas_citra, cv2.IMREAD_GRAYSCALE)
        
        # Normalisasi piksel ke rentang 0-1
        citra = citra / 255.0
        
        images.append(citra)
        
        # Ambil label dari nama berkas (contoh: "plat" atau "non_plat")
        if "plat" in nama_berkas:
            labels.append(1)  # 1 untuk plat
        else:
            labels.append(0)  # 0 untuk non-plat
    
    return np.array(images), np.array(labels)

# Memuat dataset pelatihan dan pengujian
X_train, y_train = load_dataset(direktori_pelatihan_hasil)
X_test, y_test = load_dataset(direktori_pengujian_hasil)

# Membagi dataset pelatihan menjadi pelatihan dan validasi
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Menambahkan dimensi channel untuk input CNN
X_train = X_train.reshape(X_train.shape + (1,))
X_val = X_val.reshape(X_val.shape + (1,))
X_test = X_test.reshape(X_test.shape + (1,))

# Membangun model CNN sederhana
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(500, 500, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Kompilasi model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Melatih model dengan data pelatihan dan validasi
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val))

# Evaluasi model pada data pengujian
hasil_evaluasi = model.evaluate(X_test, y_test)
print(f'Akurasi pada data pengujian: {hasil_evaluasi[1]*100:.2f}%')
